# A Toy Wave Equation

Here we build a very small Cactus application to evolve the wave equation. The main new thing we are learning
about here is the MoL (Method of Lines) thorn. This thorn abstracts the time integration of your simulation
from the physics code.

In [135]:
wavetoy_dir='/home/ejwest/ETK/CactusFW2'
sim_dir='/home/ejwest/ETK/simulations'
%cd {wavetoy_dir}

/home/ejwest/ETK/CactusFW2


## 1. Create thorn, boilerplate

We're going through the boilerplate of making a new thorn, but we'll do some special
things along the way.

In [26]:
# Define some basic parameters describing a new thorn
thorn_pars = {
   "thorn_name" : "WaveToyET17",
   "arrangement_name" : "FunwaveUtils",
   "author" : "Eric West",
   "email" : "ewest@d.umn.edu",
   "license" : "BSD"
 }
import os
os.environ["ARR"]=thorn_pars["arrangement_name"]
os.environ["THORN"]=thorn_pars["thorn_name"]

This next command is only needed if you want to delete the thorn you are
going to create below and start over.

In [27]:
!rm -rf arrangements/$ARR/$THORN

Next we define some functions to help us create files automatically.

In [28]:
import re
# This function does substitutes all occurances of "{name}" in input_str with values["name"]
# and returns the new string.
def replace_values(input_str,values):
    while True:
        g = re.search(r'{(\w+)}',input_str)
        if g:
            var = g.group(1)
            if var in values:
                val = values[var]
            else:
                raise Exception("Undefined: <<"+var+">>")
            start = g.start(0)
            end = start+len(g.group(0))
            input_str = input_str[0:start]+val+input_str[end:]
            continue
        break
    return input_str

# Creates a file with the given name, uses replace_values()
# to update file_contents with file_values, and writes it out.
def create_file(file_name,file_contents,file_values):
    fd = open(file_name,"w")
    file_contents = replace_values(file_contents,file_values)
    print("Over-writing file '"+file_name+"'")
    file_contents = re.sub(r'^\s+','',file_contents)
    fd.write(file_contents)
    fd.close()
    
# The equivalent of mkdir -p
def create_dir(dir):
    print("Ensuring directory '"+dir+"'")
    if not os.path.exists(dir):
        os.makedirs(dir)

### Something new!

In the interface.ccl file we not only declare a grid function, but we give it scratch space of three time levels.
This gives MoL something to work with. Different time integrators will need different numbers of time levels.

Perhaps also unexpected are the rhs variables. The phi_rhs will hold the time derivative of phi, the psi_rhs will
hold the time derivative of psi. MoL will call the evolution routine repeatedly, storing the intermediate steps as
necessary for its time integration.

But wait! There's more! We also declare that we require a method called "MoLRegisterEvolvedGroup"
be provided by some thorn.

In [29]:
interface_ccl_contents = """
## Interface definitions for thorn {thorn_name}
inherits: grid
## An implementation name is required for all thorns. No
## two thorns in a configuration can implement the same
## interface.
implements: {thorn_name}

## the groups declared below can be public, private, or protected.
public:

## A group defines a set of variables that are allocated together
## and share common properties, i.e. timelevels, tags such as the
## Prolongation=None tag. The type tag can take on the values
## GF, Scalar, or Array.

## Note that the number of timelevels can be an integer parameter
## GF stands for "Grid Function" and refers to a distributed array
## data structure.
#cctk_real force_group type=GF timelevels=3 tags='Prolongation="None"'
#{
#  force1, force2
#}

## Scalars are single variables that are available on all processors.
#cctk_real scalar_group type=SCALAR 
#{
#  scalar1, scalar2
#}

cctk_real evol_group type=GF timelevels=3
{
  psi, phi
}
cctk_real rhs_group type=GF 
{
  psi_rhs, phi_rhs
}

#----------------------------------------------------
#   MoL functions to register variables
#----------------------------------------------------

CCTK_INT FUNCTION MoLRegisterEvolvedGroup(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
REQUIRES FUNCTION MoLRegisterEvolvedGroup

"""

### Something new!

We schedule three routines. The initial and evolution routines are unsurprising. The register method, however, is
new. This is the place where we will tell MoL what variables we are evolving.

In [30]:
schedule_ccl_contents = """
## Schedule definitions for thorn {thorn_name}

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## These commented examples correspond to the commented
## examples in the interface file above.
# storage: force_group[3]
# storage: scalar_group

## Schedule a function defined in this thorn to run at the beginning
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).
#SCHEDULE init_function at CCTK_INIT
#{
#  LANG: C
#}"Do some initial stuff"

storage: evol_group[3], rhs_group

SCHEDULE init_wave_toy in CCTK_INITIAL
{
  LANG: C
} "initialize data"

SCHEDULE evolve_wave_toy in MoL_CalcRHS
{
  LANG: C
} "evolve data"

schedule wave_toy_registervars in MoL_Register
{
  LANG: C
}"Register funwave variables for MoL"

"""

In [31]:
param_ccl_contents = """
## Parameter definitions for thorn {thorn_name}
## There are five types of parameters: int, real, keyword, string, and boolean.
## The comments provide prototypes of each.
#
#CCTK_INT one_to_five "This integer parameter goes from 1 to 5"
#{
#  1:5 :: "Another comment"
#} 3 # This is the default value
#
#CCTK_REAL from_2p5_to_3p8e4 "This integer parameter goes from 2.5 to 3.8e4"
#{
#  2.5:3.8e4 :: "Another comment"
#} 4.4e3 # This is the default value
#
## This keyword example defines the parameter wavemaker_type and 8 possible values.
#CCTK_KEYWORD wavemaker_type "types of wave makers"
#{
#  "ini_rec" :: "initial rectangular hump, need xc,yc and wid"
#  "lef_sol" :: "initial solitary wave, WKN B solution, need amp, dep"
#  "ini_oth" :: "other initial distribution specified by users"
#  "wk_reg" :: "Wei and Kirby 1999 internal wave maker, need xc_wk, tperiod, amp_wk, dep_wk, theta_wk, and time_ramp (factor of period)"
#  "wk_irr" :: "Wei and Kirby 1999 TMA spectrum wavemaker, need xc_wk, dep_wk, time_ramp, delta_wk, freqpeak, freqmin, freqmax, hmo, gammatma, theta-peak"
#  "wk_time_series" :: "fft a time series to get each wave component and then use Wei and Kirby's ( 1999) wavemaker. Need input wavecompfile (including 3 columns:   per,amp,pha) and numwavecomp, peakperiod, dep_wk, xc_wk, ywidth_wk"
#  "ini_gau" :: "initial Gaussian hump, need amp, xc, yc, and wid"
#  "ini_sol" :: "initial solitary wave, xwavemaker"
#}"wk_reg" # This is the default value
#
#CCTK_STRING a_string_par "a comment"
#{
#  .* :: "This is a perl 5 regular expression defining what the string may contain"
#} "blah blah blah" # This is the default value
#
#BOOLEAN a_boolean_par "a comment"
#{
#} true


"""

In [32]:
configuration_ccl_contents = """
# Configuration definitions for thorn {thorn_name}
## You should not need include "mpi.h", but if you
## do, you will need this next line.
# REQUIRES MPI
# REQUIRES HDF5
"""

In [33]:
makefile_contents = """
# Main make.code.defn file for thorn {thorn_name}

# Source files in this directory
SRCS = init.cc evolve.cc molregister.cc

# Subdirectories containing source files
SUBDIRS =
"""

In [34]:
readme_contents = """
Author(s)    : {author} <{email}>
Maintainer(s): {author} <{email}>
Licence      : {license}
--------------------------------------------------------------------------

1. Purpose

not documented
"""

In [35]:
import os

# This function will create a complete thorn
def create_thorn():
  # Create the thorn directory inside the Cactus source tree
  arrangement_dir = "arrangements/"+thorn_pars["arrangement_name"]
  thorn_dir = arrangement_dir + "/" + thorn_pars["thorn_name"]
  create_dir(thorn_dir)

  # Create basic ccl files needed by Cactus
  schedule_ccl = thorn_dir+"/schedule.ccl"
  interface_ccl = thorn_dir+"/interface.ccl"
  param_ccl = thorn_dir+"/param.ccl"
  configuration_ccl = thorn_dir+"/configuration.ccl"
  create_file(schedule_ccl,schedule_ccl_contents,thorn_pars)
  create_file(interface_ccl,interface_ccl_contents,thorn_pars)
  create_file(param_ccl,param_ccl_contents,thorn_pars)
  create_file(configuration_ccl,configuration_ccl_contents,thorn_pars)

  # Create the source directory and first makefile
  src_dir = thorn_dir+"/src"
  create_dir(src_dir)
  create_file(src_dir+"/make.code.defn",makefile_contents,thorn_pars)

  # Other dirs and files not strictly needed
  # for compiling and running Cactus.
  create_file(thorn_dir+"/README",readme_contents,thorn_pars)
  
  test_dir = thorn_dir+"/test"
  create_dir(test_dir)
  par_dir = thorn_dir+"/par"
  create_dir(par_dir)
  doc_dir = thorn_dir+"/doc"
  create_dir(doc_dir)

In [36]:
create_thorn()

Ensuring directory 'arrangements/FunwaveUtils/WaveToyET17'
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/schedule.ccl'
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/interface.ccl'
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/param.ccl'
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/configuration.ccl'
Ensuring directory 'arrangements/FunwaveUtils/WaveToyET17/src'
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/src/make.code.defn'
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/README'
Ensuring directory 'arrangements/FunwaveUtils/WaveToyET17/test'
Ensuring directory 'arrangements/FunwaveUtils/WaveToyET17/par'
Ensuring directory 'arrangements/FunwaveUtils/WaveToyET17/doc'


In [125]:
my_thorns_contents="""
# ./configs/sim/ThornList
# This file was automatically generated using the GetComponents script.

!CRL_VERSION = 2.0


# Component list: funwave.th

!DEFINE ROOT = CactusFW2
!DEFINE ARR = $ROOT/arrangements
!DEFINE ET_RELEASE = trunk
!DEFINE FW_RELEASE = FW_2014_05

#Cactus Flesh
!TARGET   = $ROOT
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactus.git
!NAME     = flesh
!CHECKOUT = CONTRIBUTORS COPYRIGHT doc lib Makefile src

!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/stevenrbrandt/cajunwave.git
!REPO_PATH= $2
# Old version
#!AUTH_URL = https://svn.cct.lsu.edu/repos/projects/ngchc/code/branches/$FW_RELEASE/$1/$2
#!URL = https://svn.cct.lsu.edu/repos/projects/ngchc/code/branches/$FW_RELEASE/$2
#!URL = https://svn.cct.lsu.edu/repos/projects/ngchc/code/CactusCoastal/$2
!CHECKOUT =
CactusCoastal/Funwave
CactusCoastal/FunwaveMesh
CactusCoastal/FunwaveCoord
CactusCoastal/Tridiagonal
CactusCoastal/Tridiagonal2

# CactusBase thorns
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusbase.git
!REPO_PATH= $2
!CHECKOUT =
CactusBase/Boundary
CactusBase/CartGrid3D
CactusBase/CoordBase
CactusBase/Fortran
CactusBase/InitBase
CactusBase/IOASCII
CactusBase/IOBasic
CactusBase/IOUtil
CactusBase/SymBase
CactusBase/Time
#
# CactusNumerical thorns
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusnumerical.git
!REPO_PATH= $2
!CHECKOUT =
!CHECKOUT =
CactusNumerical/MoL
CactusNumerical/LocalInterp

CactusNumerical/Dissipation
CactusNumerical/SpaceMask
CactusNumerical/SphericalSurface
CactusNumerical/LocalReduce
CactusNumerical/InterpToArray

!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusutils.git
!REPO_PATH= $2
!CHECKOUT = CactusUtils/Accelerator CactusUtils/OpenCLRunTime
CactusUtils/NaNChecker
CactusUtils/Vectors
CactusUtils/SystemTopology

# Carpet, the AMR driver
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/eschnett/carpet.git
!REPO_PATH= $2
!CHECKOUT = Carpet/doc
Carpet/Carpet
Carpet/CarpetEvolutionMask
Carpet/CarpetIOASCII
Carpet/CarpetIOBasic
Carpet/CarpetIOHDF5
Carpet/CarpetIOScalar
#Carpet/CarpetIntegrateTest
Carpet/CarpetInterp
Carpet/CarpetInterp2
Carpet/CarpetLib
Carpet/CarpetMask
#Carpet/CarpetProlongateTest
Carpet/CarpetReduce
Carpet/CarpetRegrid
Carpet/CarpetRegrid2
#Carpet/CarpetRegridTest
Carpet/CarpetSlab
Carpet/CarpetTracker
Carpet/CycleClock
#Carpet/HighOrderWaveTest
Carpet/LoopControl
#Carpet/ReductionTest
#Carpet/ReductionTest2
#Carpet/ReductionTest3
#Carpet/RegridSyncTest
Carpet/TestCarpetGridInfo
Carpet/TestLoopControl
Carpet/Timers

# Additional Cactus thorns
!TARGET   = $ARR
!TYPE     = svn
!URL      = https://svn.cactuscode.org/projects/$1/$2/trunk
!CHECKOUT = ExternalLibraries/OpenBLAS ExternalLibraries/OpenCL ExternalLibraries/pciutils ExternalLibraries/PETSc
ExternalLibraries/MPI
ExternalLibraries/HDF5
ExternalLibraries/zlib
ExternalLibraries/hwloc

# Simulation Factory
!TARGET   = $ROOT/simfactory
!TYPE     = git
!URL      = https://bitbucket.org/simfactory/simfactory2.git
!NAME     = simfactory2
!CHECKOUT = README.md README_FIRST.txt bin doc etc lib mdb

# Various thorns from LSU
#!TARGET   = $ARR
#!TYPE     = git
#!URL      = https://bitbucket.org/einsteintoolkit/archivedthorns-vectors.git
#!REPO_PATH= $2
#!CHECKOUT =
#LSUThorns/Vectors
#LSUThorns/QuasiLocalMeasures
#LSUThorns/SummationByParts
#LSUThorns/Prolong

#Roland/MapPoints
#Tutorial/BadWaveMoL
#Tutorial/BasicWave
#Tutorial/BasicWave2
#Tutorial/BasicWave3

# Various thorns from the AEI
# Numerical
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/numerical.git
!REPO_PATH= $2
!CHECKOUT =
#AEIThorns/ADMMass
AEIThorns/AEILocalInterp
#AEIThorns/PunctureTracker
#AEIThorns/SystemStatistics
#AEIThorns/Trigger
{arrangement_name}/{thorn_name}"""
%mkdir -p thornlists
create_file("thornlists/my_thorns_wavetoy.th",my_thorns_contents,thorn_pars)

Over-writing file 'thornlists/my_thorns_wavetoy.th'


## 2. Write thorn code

### Something new!

Here we create the source code. The initialization, evolution, and molregister functions.

In [145]:
%cd {wavetoy_dir}

/home/ejwest/ETK/CactusFW2


In [146]:
src_dir = "arrangements/"+thorn_pars["arrangement_name"]+"/"+thorn_pars["thorn_name"]+"/src"

init_cc_src = """
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <math.h>

void init_wave_toy(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  for(int k=0;k<cctk_lsh[2];k++) {
      for(int j=0;j<cctk_lsh[1];j++) {
          for(int i=0;i<cctk_lsh[0];i++) {
              int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
              CCTK_REAL shift_x = 15.0;
              CCTK_REAL shift_y = 15.0;
              CCTK_REAL xx = x[cc] - shift_x;
              CCTK_REAL yy = y[cc] - shift_y;
              psi[cc] = exp(-xx*xx-yy*yy);
              phi[cc] = 0;
          }
      }
  }
}
"""
create_file(src_dir+"/init.cc",init_cc_src,{})

Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/src/init.cc'


In [147]:
evolve_cc_src = """
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <math.h>
#include <assert.h>

void evolve_wave_toy(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  for(int k=0;k<cctk_lsh[2];k++) {
      for(int j=1;j<cctk_lsh[1]-1;j++) {
          for(int i=1;i<cctk_lsh[0]-1;i++) {
              int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
              int im1 = CCTK_GFINDEX3D(cctkGH,i-1,j,k);
              int ip1 = CCTK_GFINDEX3D(cctkGH,i+1,j,k);
              int jm1 = CCTK_GFINDEX3D(cctkGH,i,j-1,k);
              int jp1 = CCTK_GFINDEX3D(cctkGH,i,j+1,k);
              CCTK_REAL dx = CCTK_DELTA_SPACE(0);
              CCTK_REAL dy = CCTK_DELTA_SPACE(1);
              psi_rhs[cc] = phi[cc];
              CCTK_REAL divx = (psi[ip1] + psi[im1] - 2*psi[cc])/(dx*dx);
              CCTK_REAL divy = (psi[jp1] + psi[jm1] - 2*psi[cc])/(dy*dy);
              phi_rhs[cc] = divx + divy;
          }
      }
  }
}
"""
create_file(src_dir+"/evolve.cc",evolve_cc_src,{})

Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/src/evolve.cc'


In [148]:
molregister_cc_src = """
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>

extern "C" void wave_toy_registervars (CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  CCTK_INT ierr = 0, group, rhs;

  // Look up the group index of the evol_group
  group = CCTK_GroupIndex ("{thorn_name}::evol_group");
  
  // Look up the group index of the rhs_group
  rhs = CCTK_GroupIndex ("{thorn_name}::rhs_group");

  if (CCTK_IsFunctionAliased ("MoLRegisterEvolvedGroup"))
  {
    // *** KEY STEP! *** Tell MoL what we variables
    // we are evolving and what the rhs variables are.
    ierr += MoLRegisterEvolvedGroup (group, rhs);
  }
  else
  {
    CCTK_WARN (0, "MoL function not aliased !");
    ierr++;
  }

  if (ierr)
    CCTK_WARN (0, "Problems registering with MoL !");
}

"""
create_file(src_dir+"/molregister.cc",molregister_cc_src,thorn_pars)

Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/src/molregister.cc'


In [149]:
%cd {wavetoy_dir}
!time ./simfactory/bin/sim build -j2 --thornlist=thornlists/my_thorns_wavetoy.th

/home/ejwest/ETK/CactusFW2
Using configuration: sim
Updated thorn list for configuration sim
Building sim
Cactus - version: 4.2.3
Building configuration sim
Reconfiguring thorns
Reading ThornList...
Parsing configuration files...
   Boundary
           Provides:          Boundary 
   Cactus
           Optional:          MPI 
   Carpet
           Provides:          Carpet 
           Requires:          CarpetLib IOUtil MPI Timers 
           Optional:          LoopControl 
   CarpetEvolutionMask
           Requires:          Carpet 
   CarpetIOASCII
           Requires:          Carpet CarpetLib IOUtil 
           Optional:          SDF 
   CarpetIOBasic
           Requires:          Carpet CarpetLib IOUtil 
   CarpetIOHDF5
           Provides:          CarpetIOHDF5 
           Requires:          Carpet CarpetLib IOUtil HDF5 
           Optional:          MPI 
   CarpetIOScalar
           Requires:          Carpet CarpetLib IOUtil 
   CarpetInterp
           Requires:          Carpet Ca

Parsing parameter files...
   AEILocalInterp
          0 Global, 0 Restricted, 1 Private parameters
   Boundary
          0 Global, 8 Restricted, 0 Private parameters
   Cactus
          0 Global, 6 Restricted, 13 Private parameters
   Carpet
          0 Global, 4 Restricted, 94 Private parameters
   CarpetEvolutionMask
          0 Global, 0 Restricted, 6 Private parameters
   CarpetIOASCII
          0 Global, 0 Restricted, 63 Private parameters
   CarpetIOBasic
          0 Global, 0 Restricted, 15 Private parameters
   CarpetIOHDF5
          0 Global, 0 Restricted, 77 Private parameters
   CarpetIOScalar
          0 Global, 0 Restricted, 9 Private parameters
   CarpetInterp
          0 Global, 0 Restricted, 5 Private parameters
   CarpetInterp2
          0 Global, 0 Restricted, 3 Private parameters
   CarpetLib
          0 Global, 5 Restricted, 31 Private parameters
   CarpetMask
          0 Global, 0 Restricted, 8 Private parameters
   CarpetReduce
          0 Global, 0 Restricted, 3

Checking status of thorn CarpetIOASCII
Checking status of thorn CarpetIOBasic
Checking status of thorn HDF5
Checking status of thorn CarpetIOHDF5
Checking status of thorn CarpetIOScalar
Checking status of thorn CarpetInterp
Checking status of thorn CarpetInterp2
Checking status of thorn CarpetMask
Checking status of thorn CarpetReduce
Checking status of thorn CarpetRegrid
Checking status of thorn CarpetRegrid2
Checking status of thorn CarpetSlab
Checking status of thorn CarpetTracker
Checking status of thorn CoordBase
Checking status of thorn CartGrid3D
Checking status of thorn SphericalSurface
Checking status of thorn Dissipation
Checking status of thorn Fortran
Checking status of thorn Funwave
Checking status of thorn FunwaveCoord
Checking status of thorn FunwaveMesh
Checking status of thorn IOASCII
Checking status of thorn IOBasic
Checking status of thorn InitBase
Checking status of thorn InterpToArray
Checking status of thorn LocalInterp
Checking status of thorn LocalReduce
Checkin

## 3. Run

In [164]:
# A minimal par file....

wave_toy_par_src = """
ActiveThorns = "{thorn_name} CoordBase Carpet CartGrid3d MoL"

Cactus::cctk_itlast = 250

# the output dir will be named after the parameter file name
IO::out_dir = $parfile

# The set of parameters required to specify the grid...
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

# Because we're doing a 2D code...
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# The time integration scheme...
MoL::ODE_Method = "RK3"

# The next section is required if you want some kind of confirmation
# on the console that your code is actually doing anything...
ActiveThorns = "CarpetIOBasic IOUtil CarpetReduce SymBase"
IO::out_fileinfo="none"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "{thorn_name}::phi {thorn_name}::psi"

# Get some data!
ActiveThorns = "CarpetIOASCII"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 25
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "{thorn_name}::psi {thorn_name}::phi"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

# Taking too big a time step can lead to unstable evolution.
# You can control that with dtfac and thorn Time.
ActiveThorns = "Time"
Time::dtfac = .2
"""
%mkdir -p par
create_file("par/wave_toy.par",wave_toy_par_src,thorn_pars)

Over-writing file 'par/wave_toy.par'


In [206]:
%rm -rf {sim_dir}/wave_toy
%cd {wavetoy_dir}
!./simfactory/bin/sim create-run wave_toy --parfile=par/wave_toy.par --procs=2

/home/ejwest/ETK/CactusFW2
Parameter file: /home/ejwest/ETK/CactusFW2/par/wave_toy.par
Skeleton Created
Job directory: "/home/ejwest/ETK/simulations/wave_toy"
Executable: "/home/ejwest/ETK/CactusFW2/exe/cactus_sim"
Option list: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/cfg/OptionList"
Submit script: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/run/SubmitScript"
Run script: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/run/RunScript"
Parameter file: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/par/wave_toy.par"
Simulation name: wave_toy
Assigned restart id: 0 
Running simulation wave_toy
Preparing:
+ set -e
+ cd /home/ejwest/ETK/simulations/wave_toy/output-0000-active
+ echo Checking:
Checking:
+ pwd
/home/ejwest/ETK/simulations/wave_toy/output-0000-active
+ hostname
newton
+ date
Mon Apr 29 16:10:34 CDT 2019
+ echo Environment:
Environment:
+ export CACTUS_NUM_PROCS=1
+ export CACTUS_NUM_THREADS=2
+ export GMON_OUT_PREFIX=gmon.out
+ export OMP_NUM_THREADS=2
+ env


INFO (CycleClock): Calibrated CycleClock: 0.303711 ns per clock tick (3.29261 GHz)
INFO (Vectors): Using vector size 2 for architecture SSE2 (64-bit precision)
--------------------------------------------------------------------------------
AMR driver provided by Carpet
--------------------------------------------------------------------------------
AMR info I/O provided by CarpetIOBasic
--------------------------------------------------------------------------------
MoL: Generalized time integration.
--------------------------------------------------------------------------------
AMR 0D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 1D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 2D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 3D ASCII I/O provided by CarpetIOASC

       39     1.950 |   -0.2218747    0.2012299 |   -0.3877370    0.2386950
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
       40     2.000 |   -0.2113639    0.1996998 |   -0.3845668    0.2371067
       41     2.050 |   -0.2021501    0.1969922 |   -0.3767959    0.2323288
       42     2.100 |   -0.1943005    0.1949926 |   -0.3707725    0.2250242
       43     2.150 |   -0.1871954    0.1941830 |   -0.3695076    0.2157991
       44     2.200 |   -0.1811315    0.1919436 |   -0.3637509    0.2051961
       45     2.250 |   -0.1753969    0.1892964 |   -0.3563209    0.1936904
       46     2.300 |   -0.1704707    0.1884530 |   -0.3536325    0.1816882
       47     2.350 |   -0.1657751    0.1872293 |   -0.3513250    0.1702944
       48   

      145     7.250 |   -0.0699562    0.1123697 |   -0.1920686    0.0968938
      146     7.300 |   -0.0701647    0.1123084 |   -0.1925462    0.0958858
      147     7.350 |   -0.0698758    0.1116016 |   -0.1916355    0.0961843
      148     7.400 |   -0.0695054    0.1109192 |   -0.1903557    0.0961556
      149     7.450 |   -0.0691443    0.1111464 |   -0.1899003    0.0952010
      150     7.500 |   -0.0687883    0.1107149 |   -0.1899813    0.0946514
      151     7.550 |   -0.0689978    0.1100069 |   -0.1887709    0.0951535
      152     7.600 |   -0.0687411    0.1097203 |   -0.1876254    0.0946056
      153     7.650 |   -0.0683856    0.1097563 |   -0.1878751    0.0936281
      154     7.700 |   -0.0680369    0.1091142 |   -0.1872211    0.0937924
      155     7.750 |   -0.0676953    0.1084580 |   -0.1860525    0.0938703
      156     7.800 |   -0.0679055    0.1086200 |   -0.1852059    0.0929965
      157     7.850 |   -0.0676812    0.1082838 |   -0.1856448    0.0922846
      158   

      238    11.900 |   -0.0566337    0.0876585 |   -0.1493503    0.0732634
      239    11.950 |   -0.0564300    0.0873724 |   -0.1498186    0.0732762
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
      240    12.000 |   -0.0562006    0.0869700 |   -0.1492855    0.0734717
      241    12.050 |   -0.0560133    0.0869134 |   -0.1486192    0.0729617
      242    12.100 |   -0.0562480    0.0869196 |   -0.1484452    0.0723216
      243    12.150 |   -0.0561947    0.0865205 |   -0.1485170    0.0728379
      244    12.200 |   -0.0559916    0.0861232 |   -0.1479008    0.0726711
      245    12.250 |   -0.0557738    0.0863128 |   -0.1472497    0.0720594
      246    12.300 |   -0.0555875    0.0860881 |   -0.1475617    0.0719583
      247   

In [207]:
# This cell enables inline plotting in the notebook
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation

In [208]:
%cd {sim_dir}/wave_toy/output-0000/wave_toy
%ls

/home/ejwest/ETK/simulations/wave_toy/output-0000/wave_toy
phi.xy.asc  psi.xy.asc  wave_toy.par


In [209]:
# animate xy-slice using imshow()
file_data = np.genfromtxt('psi.xy.asc')

# figure properties
cmap = cm.prism
fig_width = 4
fig_height = 4
minval = np.min(file_data[:,12])
maxval = np.max(file_data[:,12])

# time steps
time_steps = np.unique(file_data[:,0]) 

# extract data
def extract_data(tstep): 
    ttdata = file_data[file_data[:,0]==tstep, :] #find matching time blocks
    x = ttdata[:, 5]  #x coordinate in 5th column
    y = ttdata[:, 6]  #y coordinate in 6th column
    z = ttdata[:, 12] #data in 12th column
    zi = z.reshape(len(np.unique(y)), len(np.unique(x))) #reshape
    return zi[::-1, :]

# initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_subplot(111)
data = extract_data(time_steps[0])
plot = [ax.imshow(data, cmap=cmap, clim=(minval, maxval))]

# update plot
def update_plot(frame_number, extract_data, plot):
    plot[0].remove()
    data = extract_data(time_steps[frame_number])
    plot[0] = ax.imshow(data, cmap=cmap, clim=(minval, maxval))

# make animation
fps=3
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(extract_data, plot), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

# play animation
from IPython.display import HTML
HTML(anim.to_jshtml())

## 4. Multiple procs

### Question: 
Does the above simulation work properly if we use 2 procs? Why or why not? Try it.

In [59]:
schedule_ccl_contents = """
## Schedule definitions for thorn {thorn_name}

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## These commented examples correspond to the commented
## examples in the interface file above.
# storage: force_group[3]
# storage: scalar_group

## Schedule a function defined in this thorn to run at the beginning
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).
#SCHEDULE init_function at CCTK_INIT
#{
#  LANG: C
#}"Do some initial stuff"

storage: evol_group[3], rhs_group

SCHEDULE init_wave_toy in CCTK_INITIAL
{
  LANG: C
} "initialize data"

SCHEDULE evolve_wave_toy in MoL_CalcRHS
{
  LANG: C
  SYNC: {thorn_name}::evol_group
  SYNC: {thorn_name}::rhs_group
} "evolve data"

schedule wave_toy_registervars in MoL_Register
{
  LANG: C
}"Register funwave variables for MoL"

"""
%cd {wavetoy_dir}
thorn_dir = "arrangements/"+thorn_pars["arrangement_name"]+"/"+thorn_pars["thorn_name"]
create_file(thorn_dir+"/schedule.ccl",schedule_ccl_contents,thorn_pars)

/home/ejwest/ETK/CactusFW2
Over-writing file 'arrangements/FunwaveUtils/WaveToyET17/schedule.ccl'


In [60]:
!time ./simfactory/bin/sim build -j2 --thornlist=thornlists/my_thorns_wavetoy.th

Using configuration: sim
Updated thorn list for configuration sim
Building sim
Cactus - version: 4.2.3
Building configuration sim
Reconfiguring thorns
Reading ThornList...
Parsing configuration files...
   Boundary
           Provides:          Boundary 
   Cactus
           Optional:          MPI 
   Carpet
           Provides:          Carpet 
           Requires:          CarpetLib IOUtil MPI Timers 
           Optional:          LoopControl 
   CarpetEvolutionMask
           Requires:          Carpet 
   CarpetIOASCII
           Requires:          Carpet CarpetLib IOUtil 
           Optional:          SDF 
   CarpetIOBasic
           Requires:          Carpet CarpetLib IOUtil 
   CarpetIOHDF5
           Provides:          CarpetIOHDF5 
           Requires:          Carpet CarpetLib IOUtil HDF5 
           Optional:          MPI 
   CarpetIOScalar
           Requires:          Carpet CarpetLib IOUtil 
   CarpetInterp
           Requires:          Carpet CarpetLib MPI 
   CarpetInter

   AEILocalInterp
          0 Global, 0 Restricted, 1 Private parameters
   Boundary
          0 Global, 8 Restricted, 0 Private parameters
   Cactus
          0 Global, 6 Restricted, 13 Private parameters
   Carpet
          0 Global, 4 Restricted, 94 Private parameters
   CarpetEvolutionMask
          0 Global, 0 Restricted, 6 Private parameters
   CarpetIOASCII
          0 Global, 0 Restricted, 63 Private parameters
   CarpetIOBasic
          0 Global, 0 Restricted, 15 Private parameters
   CarpetIOHDF5
          0 Global, 0 Restricted, 77 Private parameters
   CarpetIOScalar
          0 Global, 0 Restricted, 9 Private parameters
   CarpetInterp
          0 Global, 0 Restricted, 5 Private parameters
   CarpetInterp2
          0 Global, 0 Restricted, 3 Private parameters
   CarpetLib
          0 Global, 5 Restricted, 31 Private parameters
   CarpetMask
          0 Global, 0 Restricted, 8 Private parameters
   CarpetReduce
          0 Global, 0 Restricted, 3 Private parameters
   Carp

Checking status of thorn CarpetIOBasic
Checking status of thorn HDF5
Checking status of thorn CarpetIOHDF5
Checking status of thorn CarpetIOScalar
Checking status of thorn CarpetInterp
Checking status of thorn CarpetInterp2
Checking status of thorn CarpetMask
Checking status of thorn CarpetReduce
Checking status of thorn CarpetRegrid
Checking status of thorn CarpetRegrid2
Checking status of thorn CarpetSlab
Checking status of thorn CarpetTracker
Checking status of thorn CoordBase
Checking status of thorn CartGrid3D
Checking status of thorn SphericalSurface
Checking status of thorn Dissipation
Checking status of thorn Fortran
Checking status of thorn Funwave
Checking status of thorn FunwaveCoord
Checking status of thorn FunwaveMesh
Checking status of thorn IOASCII
Checking status of thorn IOBasic
Checking status of thorn InitBase
Checking status of thorn InterpToArray
Checking status of thorn LocalInterp
Checking status of thorn LocalReduce
Checking status of thorn MoL
Checking status o

In [61]:
%rm -rf {sim_dir}/wave_toy
%cd {wavetoy_dir}
!./simfactory/bin/sim create-run wave_toy --par=par/wave_toy.par --procs=2 --num-threads=1

Parameter file: /home/ejwest/ETK/CactusFW2/par/wave_toy.par
Skeleton Created
Job directory: "/home/ejwest/ETK/simulations/wave_toy"
Executable: "/home/ejwest/ETK/CactusFW2/exe/cactus_sim"
Option list: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/cfg/OptionList"
Submit script: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/run/SubmitScript"
Run script: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/run/RunScript"
Parameter file: "/home/ejwest/ETK/simulations/wave_toy/SIMFACTORY/par/wave_toy.par"
Simulation name: wave_toy
Assigned restart id: 0 
Running simulation wave_toy
Preparing:
+ set -e
+ cd /home/ejwest/ETK/simulations/wave_toy/output-0000-active
+ echo Checking:
Checking:
+ pwd
/home/ejwest/ETK/simulations/wave_toy/output-0000-active
+ hostname
newton
+ date
Mon Apr 29 11:44:34 CDT 2019
+ echo Environment:
Environment:
+ export CACTUS_NUM_PROCS=2
+ export CACTUS_NUM_THREADS=1
+ export GMON_OUT_PREFIX=gmon.out
+ export OMP_NUM_THREADS=1
+ env
+ sort
+ echo Starting:
Sta

INFO (CycleClock): Calibrated CycleClock: 0.303714 ns per clock tick (3.29257 GHz)
INFO (Vectors): Using vector size 2 for architecture SSE2 (64-bit precision)
--------------------------------------------------------------------------------
AMR driver provided by Carpet
--------------------------------------------------------------------------------
AMR info I/O provided by CarpetIOBasic
--------------------------------------------------------------------------------
MoL: Generalized time integration.
--------------------------------------------------------------------------------
AMR 0D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 1D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 2D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 3D ASCII I/O provided by CarpetIOASC

       49     2.450 |   -0.1579006    0.1831797 |   -0.3401713    0.1566642
       50     2.500 |   -0.1540334    0.1826335 |   -0.3396896    0.1543173
       51     2.550 |   -0.1501820    0.1807414 |   -0.3350467    0.1536697
       52     2.600 |   -0.1476378    0.1785308 |   -0.3286432    0.1535596
       53     2.650 |   -0.1444369    0.1777614 |   -0.3270033    0.1516594
       54     2.700 |   -0.1415830    0.1767957 |   -0.3254103    0.1494606
       55     2.750 |   -0.1392927    0.1748061 |   -0.3199680    0.1498607
       56     2.800 |   -0.1366551    0.1731982 |   -0.3158561    0.1491010
       57     2.850 |   -0.1338915    0.1728732 |   -0.3160229    0.1469808
       58     2.900 |   -0.1319841    0.1712680 |   -0.3122932    0.1462140
       59     2.950 |   -0.1299540    0.1693620 |   -0.3067014    0.1463204
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
            

      151     7.550 |   -0.2266350    0.1100069 |   -0.2788161    0.5100441
      152     7.600 |   -0.2255673    0.1097203 |   -0.2808827    0.5124807
      153     7.650 |   -0.2244659    0.1097563 |   -0.2794750    0.5088935
      154     7.700 |   -0.2218925    0.1091142 |   -0.2745203    0.4993227
      155     7.750 |   -0.2206719    0.1084580 |   -0.2735087    0.4839089
      156     7.800 |   -0.2201810    0.1086200 |   -0.2757359    0.4747648
      157     7.850 |   -0.2182747    0.1082838 |   -0.2745174    0.4651315
      158     7.900 |   -0.2157778    0.1218671 |   -0.2697716    0.4506455
      159     7.950 |   -0.2161471    0.1355825 |   -0.2704389    0.4375048
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
      160   

      248    12.400 |   -0.1626684    0.1401532 |   -0.2051643    0.2756082
      249    12.450 |   -0.1638613    0.1399784 |   -0.2072843    0.2776500
      250    12.500 |   -0.1642499    0.1410040 |   -0.2075410    0.2766080
--------------------------------------------------------------------------------
Done.
+ echo Stopping:
Stopping:
+ date
Mon Apr 29 11:44:37 CDT 2019
+ echo Done.
Done.
Mon Apr 29 11:44:37 CDT 2019
Simfactory Done at date: 0


In [62]:
%cd {sim_dir}/wave_toy/output-0000/wave_toy

/home/ejwest/ETK/simulations/wave_toy/output-0000/wave_toy


In [85]:
# animate xy-slice using imshow()
file_data = np.genfromtxt('psi.xy.asc')

# figure properties
cmap = cm.prism
fig_width = 4
fig_height = 4
minval = np.min(file_data[:,12])
maxval = np.max(file_data[:,12])

# time steps
time_steps = np.unique(file_data[:,0]) 

# extract data
def extract_data(tstep): 
    ttdata = file_data[file_data[:,0]==tstep, :] #find matching time blocks
    x = ttdata[:, 5]  #x coordinate in 5th column
    y = ttdata[:, 6]  #y coordinate in 6th column
    z = ttdata[:, 12] #data in 12th column
    zi = z.reshape(len(np.unique(y)), len(np.unique(x))) #reshape
    return zi[::-1, :]

# initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_subplot(111)
data = extract_data(time_steps[0])
plot = [ax.imshow(data, cmap=cmap, clim=(minval, maxval))]

# update plot
def update_plot(frame_number, extract_data, plot):
    plot[0].remove()
    data = extract_data(time_steps[frame_number])
    plot[0] = ax.imshow(data, cmap=cmap, clim=(minval, maxval))

# make animation
fps=3
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(extract_data, plot), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

# play animation
from IPython.display import HTML
HTML(anim.to_jshtml())

### Exercise:

Add a parameter to param.ccl and use it to change the position of the Gaussian wave hump.

## 5. Adaptive Mesh Refinement

Sometimes you care more about what is happening in one part of a grid than another. Wouldn't it be nice if you could spend less time computing in regions that are less important? That way you could explore larger problems with less compute cycles.

Well, dream no more. Adaptive Mesh Refinement is supported in Cactus/Carpet. And, at least for the toy example we are considering in this file, all you have to change to make use of it is your parameter file. (Actually, what we're going to consider here is technically Fixed Mesh Refinement (FMR)).

<h3>How AMR works</h3>
For reasons of simplicity, we'll focus on a 2 refinement level system. Thus, we have a coarse (refinement level 0) and a fine level (refinement level 1). We start from a time t0, at which data on all refinement levels is consistent and valid.

* All data is initialized at time t = t0
* The coarse level is evolved forward to time to t = t0 + dt
* The fine level is evolved forward to time to t = t0 + dt/2
* Boundary data for the fine grid at time to t = t0+dt/2 is interpolated from the coarse grids at times t0 and t0+dt
* The fine level is evolved forward in time to t = t0 + dt
* Data from the fine level at t=t0+dt is copied back into the coarse grid at time t=t0+dt
* If not done, t0 += dt and repeat.

In [238]:
wave_toy2_par_src = """
ActiveThorns = "{thorn_name} coordbase carpet cartgrid3d MoL"

Cactus::cctk_itlast = 1000

# the output dir will be named after the parameter file name
IO::out_dir = $parfile

# The set of parameters required to specify the grid...
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

# Because we're doing a 2D code...
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# The time integration scheme...
MoL::ODE_Method = "RK3"

# The next section is required if you want some kind of confirmation
# on the console that your code is actually doing anything...
ActiveThorns = "CarpetIOBasic IOUtil CarpetReduce SymBase"
IO::out_fileinfo="all"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "{thorn_name}::phi {thorn_name}::psi"

# Get some data!
ActiveThorns = "CarpetIOASCII"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 25
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "{thorn_name}::psi {thorn_name}::phi"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

# Taking too big a time step can lead to unstable evolution.
# You can control that with dtfac and thorn Time.
ActiveThorns = "Time"
Time::dtfac = .2

### MESH REFINEMENT ####
ActiveThorns = "CarpetRegrid2"
Carpet::init_fill_timelevels = "true"
Carpet::max_refinement_levels = 3
CarpetRegrid2::num_centres = 1
CarpetRegrid2::num_levels_1 = 3
CarpetRegrid2::position_x_1 = 15.0
CarpetRegrid2::position_y_1 = 15.0
CarpetRegrid2::position_z_1 = 0.0
CarpetRegrid2::radius_1[1] = 8.0
CarpetRegrid2::radius_1[2] = 4.0

#MoL::disable_prolongation        = "no"

### Dissipation is numerical noise added to mitigate
### grid boundary effects seen in some AMR simulations.
#ActiveThorns = "Dissipation Spacemask SPHERICALSURFACE"
#Dissipation::vars = "{thorn_name}::psi {thorn_name}::phi"
#Dissipation::order = 1
"""
%cd {wavetoy_dir}
%mkdir -p par
create_file("par/wave_toy2.par",wave_toy2_par_src,thorn_pars)

/home/ejwest/ETK/CactusFW2
Over-writing file 'par/wave_toy2.par'


In [239]:
%rm -rf {sim_dir}/wave_toy2
%cd {wavetoy_dir}
!./simfactory/bin/sim create-run wave_toy2 --parfile=par/wave_toy2.par --procs=2 --num-threads=1

/home/ejwest/ETK/CactusFW2
Parameter file: /home/ejwest/ETK/CactusFW2/par/wave_toy2.par
Skeleton Created
Job directory: "/home/ejwest/ETK/simulations/wave_toy2"
Executable: "/home/ejwest/ETK/CactusFW2/exe/cactus_sim"
Option list: "/home/ejwest/ETK/simulations/wave_toy2/SIMFACTORY/cfg/OptionList"
Submit script: "/home/ejwest/ETK/simulations/wave_toy2/SIMFACTORY/run/SubmitScript"
Run script: "/home/ejwest/ETK/simulations/wave_toy2/SIMFACTORY/run/RunScript"
Parameter file: "/home/ejwest/ETK/simulations/wave_toy2/SIMFACTORY/par/wave_toy2.par"
Simulation name: wave_toy2
Assigned restart id: 0 
Running simulation wave_toy2
Preparing:
+ set -e
+ cd /home/ejwest/ETK/simulations/wave_toy2/output-0000-active
+ echo Checking:
Checking:
+ pwd
/home/ejwest/ETK/simulations/wave_toy2/output-0000-active
+ hostname
newton
+ date
Mon Apr 29 16:22:55 CDT 2019
+ echo Environment:
Environment:
+ export CACTUS_NUM_PROCS=2
+ export CACTUS_NUM_THREADS=1
+ export GMON_OUT_PREFIX=gmon.out
+ export OMP_NUM_THREA

INFO (CycleClock): Calibrated CycleClock: 0.303714 ns per clock tick (3.29257 GHz)
INFO (Vectors): Using vector size 2 for architecture SSE2 (64-bit precision)
--------------------------------------------------------------------------------
AMR driver provided by Carpet
--------------------------------------------------------------------------------
AMR info I/O provided by CarpetIOBasic
--------------------------------------------------------------------------------
MoL: Generalized time integration.
--------------------------------------------------------------------------------
AMR 0D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 1D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 2D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 3D ASCII I/O provided by CarpetIOASC

       11     0.138 | 3.693883e-196    0.9627312 |   -0.5352973    0.0724933
       12     0.150 | 3.693883e-196    0.9557525 |   -0.5811722    0.0787156
       13     0.163 | 3.693883e-196    0.9482048 |   -0.6263344    0.0848499
       14     0.175 | 3.693883e-196    0.9400973 |   -0.6707304    0.0908829
       15     0.188 | 3.693883e-196    0.9314399 |   -0.7143084    0.0968069
       16     0.200 | 3.693883e-196    0.9222432 |   -0.7570178    0.1026146
       17     0.213 | 3.693883e-196    0.9125182 |   -0.7988098    0.1082986
       18     0.225 | 3.693883e-196    0.9022769 |   -0.8396372    0.1138514
       19     0.238 | 3.693883e-196    0.8915315 |   -0.8794544    0.1192687
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
   

      101     1.263 |   -0.2447267    0.2364905 |   -0.5434775    0.1961754
      102     1.275 |   -0.2492074    0.2356571 |   -0.5359627    0.1956411
      103     1.288 |   -0.2533944    0.2347965 |   -0.5288828    0.1951135
      104     1.300 |   -0.2572929    0.2339385 |   -0.5223206    0.1946443
      105     1.312 |   -0.2609085    0.2331665 |   -0.5160619    0.1941332
      106     1.325 |   -0.2642469    0.2323451 |   -0.5102765    0.1936204
      107     1.338 |   -0.2673141    0.2315333 |   -0.5047948    0.1931090
      108     1.350 |   -0.2701160    0.2307600 |   -0.4995289    0.1926672
      109     1.363 |   -0.2726588    0.2299823 |   -0.4946891    0.1921581
      110     1.375 |   -0.2749487    0.2292035 |   -0.4900105    0.1916550
      111     1.388 |   -0.2769922    0.2284259 |   -0.4856050    0.1912106
      112     1.400 |   -0.2787956    0.2277046 |   -0.4814074    0.1907249
      113     1.413 |   -0.2803655    0.2269493 |   -0.4773337    0.1902355
      114   

      197     2.463 |   -0.1517492    0.1827557 |   -0.3332628    0.1598096
      198     2.475 |   -0.1508237    0.1823934 |   -0.3323609    0.1595013
      199     2.488 |   -0.1499181    0.1820208 |   -0.3313862    0.1592949
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
      200     2.500 |   -0.1490341    0.1816293 |   -0.3303883    0.1589875
      201     2.513 |   -0.1481583    0.1812460 |   -0.3294246    0.1586677
      202     2.525 |   -0.1472932    0.1809080 |   -0.3285617    0.1584435
      203     2.538 |   -0.1464615    0.1805262 |   -0.3275935    0.1581822
      204     2.550 |   -0.1456268    0.1801485 |   -0.3266284    0.1578646
      205     2.562 |   -0.1448113    0.1798014 |   -0.3257931    0.1575739
      206   

...

      567     7.088 |   -0.2668979    0.2160268 |   -0.4096369    0.3864618
      568     7.100 |   -0.2700070    0.2165460 |   -0.4109083    0.3852747
      569     7.113 |   -0.2734966    0.2169336 |   -0.4118894    0.3839226
      570     7.125 |   -0.2774072    0.2174350 |   -0.4125718    0.3830852
      571     7.138 |   -0.2818596    0.2180917 |   -0.4129469    0.3819776
      572     7.150 |   -0.2868503    0.2186206 |   -0.4130065    0.3806037
      573     7.163 |   -0.2920116    0.2190230 |   -0.4127427    0.3794051
      574     7.175 |   -0.2971675    0.2193000 |   -0.4121477    0.3783787
      575     7.188 |   -0.3023139    0.2194531 |   -0.4112140    0.3770794
      576     7.200 |   -0.3074464    0.2196867 |   -0.4099346    0.3755110
      577     7.213 |   -0.3125608    0.2201148 |   -0.4083025    0.3741611
      578     7.225 |   -0.3176525    0.2204228 |   -0.4063113    0.3729292
      579     7.238 |   -0.3227170    0.2206119 |   -0.4039547    0.3714201
------------

      664     8.300 |   -0.2444936    0.1728340 |   -0.2338758    0.6435254
      665     8.312 |   -0.2364093    0.1715287 |   -0.2334899    0.6498669
      666     8.325 |   -0.2282491    0.1701628 |   -0.2333755    0.6556820
      667     8.338 |   -0.2200195    0.1687359 |   -0.2332118    0.6609663
      668     8.350 |   -0.2117271    0.1672474 |   -0.2330329    0.6657165
      669     8.363 |   -0.2033788    0.1656969 |   -0.2330983    0.6699299
      670     8.375 |   -0.1949811    0.1641806 |   -0.2332890    0.6736047
      671     8.388 |   -0.1865409    0.1626159 |   -0.2334197    0.6767396
      672     8.400 |   -0.1780649    0.1609874 |   -0.2337075    0.6793344
      673     8.412 |   -0.1695598    0.1592944 |   -0.2342311    0.6813895
      674     8.425 |   -0.1610324    0.1575366 |   -0.2347931    0.6829061
      675     8.438 |   -0.1524894    0.1557134 |   -0.2353095    0.6838861
      676     8.450 |   -0.1439375    0.1538245 |   -0.2362403    0.6843322
      677   

      760     9.500 |   -0.1637276    0.2576528 |   -0.2701897    0.3641431
      761     9.513 |   -0.1670597    0.2572417 |   -0.2632548    0.3621267
      762     9.525 |   -0.1702991    0.2566433 |   -0.2561840    0.3598222
      763     9.537 |   -0.1734431    0.2558586 |   -0.2488977    0.3572345
      764     9.550 |   -0.1764888    0.2548886 |   -0.2413993    0.3543692
      765     9.562 |   -0.1794335    0.2537350 |   -0.2336922    0.3512322
      766     9.575 |   -0.1822745    0.2524870 |   -0.2259886    0.3478298
      767     9.588 |   -0.1850092    0.2510768 |   -0.2185531    0.3441687
      768     9.600 |   -0.1876349    0.2494905 |   -0.2114145    0.3403049
      769     9.613 |   -0.1901492    0.2477302 |   -0.2041002    0.3363408
      770     9.625 |   -0.1925497    0.2457981 |   -0.1966126    0.3321294
      771     9.638 |   -0.1948339    0.2436967 |   -0.2011158    0.3276788
      772     9.650 |   -0.1970360    0.2414284 |   -0.2139386    0.3229974
      773   

      855    10.688 |   -0.1432703    0.3658540 |   -0.2616875    0.4877417
      856    10.700 |   -0.1421736    0.3691269 |   -0.2643771    0.4880885
      857    10.713 |   -0.1409661    0.3723535 |   -0.2667595    0.4881153
      858    10.725 |   -0.1396464    0.3755293 |   -0.2688320    0.4878218
      859    10.738 |   -0.1382959    0.3786498 |   -0.2705925    0.4872079
---------------------------------------------------------------------------
Iteration      Time |          WAVETOYET17::psi |          WAVETOYET17::phi
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
      860    10.750 |   -0.1372682    0.3817104 |   -0.2720393    0.4874948
      861    10.763 |   -0.1361893    0.3847065 |   -0.2731710    0.4877214
      862    10.775 |   -0.1350011    0.3876336 |   -0.2739869    0.4876193
      863    10.788 |   -0.1337931    0.3904871 |   -0.2746300    0.4871878
      864   

      947    11.838 |   -0.0176268    0.2530319 |   -0.5909366    0.3657908
      948    11.850 |   -0.0174311    0.2520437 |   -0.5897792    0.3639924
      949    11.863 |   -0.0172013    0.2509211 |   -0.5881857    0.3618742
      950    11.875 |   -0.0169716    0.2500014 |   -0.5861613    0.3594379
      951    11.888 |   -0.0167000    0.2491616 |   -0.5837118    0.3573078
      952    11.900 |   -0.0164283    0.2481879 |   -0.5808436    0.3556275
      953    11.913 |   -0.0161136    0.2470829 |   -0.5775633    0.3537292
      954    11.925 |   -0.0157990    0.2461502 |   -0.5738784    0.3515106
      955    11.938 |   -0.0154403    0.2453539 |   -0.5697968    0.3489725
      956    11.950 |   -0.0150817    0.2444272 |   -0.5653268    0.3461167
      957    11.963 |   -0.0146783    0.2433729 |   -0.5604771    0.3430924
      958    11.975 |   -0.0142749    0.2425028 |   -0.5552571    0.3403049
      959    11.988 |   -0.0138263    0.2417832 |   -0.5496764    0.3377232
------------

In [240]:
%cd {sim_dir}/wave_toy2/output-0000/wave_toy2

/home/ejwest/ETK/simulations/wave_toy2/output-0000/wave_toy2


In [241]:
file_data = np.genfromtxt('psi.xy.asc')

In [216]:
time_steps = np.unique(file_data[:,0]) 
print(time_steps)

[  0.  25.  50.  75. 100. 125. 150. 175. 200. 225. 250. 275. 300. 325.
 350. 375. 400. 425. 450. 475. 500.]


In [242]:
ref_levels = np.unique(file_data[:,2])
print(ref_levels)

[0. 1. 2.]


In [243]:
centers = np.unique(file_data[:,3])
print(centers)

[0. 1.]


In [244]:
# animate xy-slice using imshow()
file_data = np.genfromtxt('psi.xy.asc')

# figure properties
cmap = cm.prism
fig_width = 4
fig_height = 4
minval = np.min(file_data[:,12])
maxval = np.max(file_data[:,12])

# time steps
time_steps = np.unique(file_data[:,0]) 

# extract data
def extract_data(tstep): 
    ttdata = file_data[file_data[:,0]==tstep, :] #find matching time blocks
    rl=0 #refinement level to plot
    rldata = ttdata[ttdata[:,2]==rl, :]
    x = rldata[:, 5]  #x coordinate in 5th column
    y = rldata[:, 6]  #y coordinate in 6th column
    z = rldata[:, 12] #data in 12th column
    zi = z.reshape(len(np.unique(y)), len(np.unique(x))) #reshape
    return zi[::-1, :]

# initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_subplot(111)
data = extract_data(time_steps[0])
plot = [ax.imshow(data, cmap=cmap, clim=(minval, maxval))]

# update plot
def update_plot(frame_number, extract_data, plot):
    plot[0].remove()
    data = extract_data(time_steps[frame_number])
    plot[0] = ax.imshow(data, cmap=cmap, clim=(minval, maxval))

# make animation
fps=3
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(extract_data, plot), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

# play animation
from IPython.display import HTML
HTML(anim.to_jshtml())

/usr/local/lib/python3.6/dist-packages/matplotlib/image.py:855: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-0.5, right=-0.5
  self.axes.set_xlim((xmin, xmax), auto=None)
/usr/local/lib/python3.6/dist-packages/matplotlib/image.py:857: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.5, top=-0.5
  self.axes.set_ylim((ymin, ymax), auto=None)


### Excercise:

Play around with the CarpetRegrid2 parameters. Attempt to add more levels of refinement and/or more centers of refinement.

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> and <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"> CCF 1539567</a></td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>